## Классификация наличия святого Георгия на изображении

In [ ]:
import os

import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torchvision
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings('ignore')

### Подгрузка данных

In [6]:
georges_df = pd.read_csv('../input_data/georges.csv', header=None)
non_georges_df = pd.read_csv('../input_data/non_georges.csv', header=None)

os.makedirs('../images/georges', exist_ok=True)
os.makedirs('../images/non_georges', exist_ok=True)

In [7]:
# Сохранение ссылок в текстовые файлы
georges_df[0].to_csv('../input_data/georges.txt', index=False, header=False)
non_georges_df[0].to_csv('../input_data/non_georges.txt', index=False, header=False)

In [ ]:
# Скачивание изображений
!wget --random-wait -i georges.txt -P images/georges
!wget --random-wait -i non_georges.txt -P images/non_georges

In [59]:
!ls images/georges | wc -l
!ls images/non_georges | wc -l

2360
3340


In [125]:
data_dir = '../images'
full_data = datasets.ImageFolder(data_dir)

labels = np.array(full_data.targets)
train_indices, val_indices = train_test_split(np.arange(len(labels)), test_size=0.2, stratify=labels, random_state=42)

# Аугментация
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = Subset(datasets.ImageFolder(data_dir, transform=train_transform), train_indices)
val_data = Subset(datasets.ImageFolder(data_dir, transform=val_transform), val_indices)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=4)

## Модель

In [127]:
model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1) 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [128]:
# Для логирования в Tensorboard
writer = SummaryWriter('../logs')

In [129]:
def save_checkpoint(state, filename):
    """Сохранение состояния модели"""
    torch.save(state, filename)


def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    best_val_acc = 0.0

    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        val_loss, val_acc = evaluate_model(model, val_loader, criterion)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}')

        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)

        if val_acc > best_val_acc:
            print(f'Validation accuracy improved from {best_val_acc:.4f} to {val_acc:.4f}. Saving model...')
            best_val_acc = val_acc

            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_loss,
                'accuracy': val_acc
            }
            save_checkpoint(checkpoint, '../checkpoints/best_checkpoint.pth')


def evaluate_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    loss = running_loss / len(loader)
    accuracy = correct / total
    return loss, accuracy

In [130]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50)

  2%|▏         | 1/50 [00:39<32:31, 39.82s/it]

Epoch [1/50], Loss: 0.6231, Accuracy: 0.6539, Val Loss: 0.6818, Val Accuracy: 0.6026
Validation accuracy improved from 0.0000 to 0.6026. Saving model...
Epoch [2/50], Loss: 0.5464, Accuracy: 0.7046, Val Loss: 0.4884, Val Accuracy: 0.7737
Validation accuracy improved from 0.6026 to 0.7737. Saving model...


  4%|▍         | 2/50 [01:19<31:55, 39.91s/it]

Epoch [3/50], Loss: 0.5148, Accuracy: 0.7397, Val Loss: 0.4370, Val Accuracy: 0.7816
Validation accuracy improved from 0.7737 to 0.7816. Saving model...


  8%|▊         | 4/50 [02:38<30:23, 39.64s/it]

...

Epoch [48/50], Loss: 0.3528, Accuracy: 0.8377, Val Loss: 0.3876, Val Accuracy: 0.7860


 98%|█████████▊| 49/50 [32:31<00:39, 39.65s/it]

Epoch [49/50], Loss: 0.3283, Accuracy: 0.8546, Val Loss: 0.3566, Val Accuracy: 0.8307


100%|██████████| 50/50 [33:11<00:00, 39.82s/it]

Epoch [50/50], Loss: 0.3395, Accuracy: 0.8441, Val Loss: 0.4463, Val Accuracy: 0.7404


![title](../tensorboard_imgs/tensorboard_accuracy.png)

![title](../tensorboard_imgs/tensorboard_loss.png)

### Инференс

In [132]:
def load_checkpoint(filename):

    checkpoint = torch.load(filename, map_location=torch.device(device))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    accuracy = checkpoint['accuracy']
    print(f"Checkpoint loaded. Epoch: {epoch}, Loss: {loss}, Accuracy: {accuracy}")
    return epoch, loss, accuracy

# Восстановление модели и оптимизатора из чекпоинта
epoch, loss, accuracy = load_checkpoint('../checkpoints/best_checkpoint.pth')

# Перевод модели в режим инференса
model.eval()

Checkpoint loaded. Epoch: 46, Loss: 0.2978761665936973, Accuracy: 0.8789473684210526


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [133]:
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
        
        outputs = model(inputs)
        sigmoid_outputs = torch.sigmoid(outputs)
        predictions = (sigmoid_outputs > 0.5).float()
        
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predictions.cpu().numpy())

In [134]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(all_labels, all_preds)
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.8105263157894737
